# Data Ingestion Pipeline - Smart Building IoT Sensors

## Casus
We gaan data verwerken van temperatuur- en CO2-sensoren in een universiteitskantoor. De sensoren sturen metingen naar verschillende bronnen:
- **Lokale JSON files** (batch uploads van sensoren)
- **CSV metadata** (sensor configuratie)
- **Publieke API** (weer data voor correlatie)

**Doel**: Inzicht krijgen in kantoorklimaat en energiebesparing mogelijk maken.

In [3]:
# Benodigde libraries
import pandas as pd
import json
import requests
from datetime import datetime, timedelta
from pathlib import Path


## 1. Trigger - Hoe en wanneer start de pipeline?

In productie zou dit een scheduled job zijn (bijv. via cron/Airflow).

**1.1 Cron expression oefening**

Hoe zou je een cron expression schrijven voor elke werkdag om 00:00?
```
* * * * *
┬ ┬ ┬ ┬ ┬
│ │ │ │ └─── Day of week (0-6 or SUN-SAT)
│ │ │ └────── Month (1-12 or JAN-DEC)
│ │ └───────── Day of month (1-31)
│ └────────────Hour (0-23)
└─────────────── Minute (0-59)
```

In [4]:
"0 0 * * 1-5"

'0 0 * * 1-5'

**1.2 Op job completion oefening**:

Als je zeker wilt weten dat een job (B) pas begint als de vorige (A) klaar is, kan je dat doen met APScheduler.

Vraag: wat gebeurt er als je de `if` statement verwijdert in `on_done()`:
```
def on_done(event):
    sched.add_job(b, id="b")
```
in plaats van 
```
def on_done(event):
    if event.job_id == "a": 
        sched.add_job(b, id="b")
```

In [5]:
"Dan zou job B ook uitgevoerd worden als job B klaar is, en dus oneindig doorgaan."

'Dan zou job B ook uitgevoerd worden als job B klaar is, en dus oneindig doorgaan.'

**1.3 Zoek JSON files** 

Controleer of er json files bestaan in FOLDER_NAME, met `Path()` en `glob()`.

In [ ]:
FOLDER_NAME = "sample_data"

**1.4 Maak een functie**

Stappen:
- Maak een functie `check_for_data()` dat op basis van een folder name de json files returnt.
- Roep de functie aan en sla het return object op in een variabele `files_to_process`

In [46]:
def check_for_data(data_dir="sample_data"):
    """Check of er JSON files zijn om te verwerken"""
    data_path = Path(data_dir)
    json_files = list(data_path.glob("*.json"))
    
    current_time = datetime.now()
    print(f"Pipeline getriggerd om {current_time.strftime('%H:%M:%S')}")
    print(f"Gevonden: {len(json_files)} JSON files in {data_dir}/")
    
    return json_files

# Test de trigger
files_to_process = check_for_data()
if files_to_process:
    print("Data beschikbaar, pipeline start!")
    for f in files_to_process:
        print(f"  - {f.name}")

Pipeline getriggerd om 14:43:36
Gevonden: 2 JSON files in sample_data/
Data beschikbaar, pipeline start!
  - sensor_readings_day2.json
  - sensor_readings_day1.json


## 2. Connection - Hoe verbindt de source met de target?

We hebben 3 verschillende databronnen voor deze oefening:
1. Lokale JSON files (sensor readings)
2. CSV file (sensor metadata)
3. Publieke API (Open-Meteo voor weer data)

In productie:
- Authentication (API token, OAuth2, service account)
- Credentials in environment variables (NOOIT in code!)
- Retry logic voor netwerkfouten

**2.1 Haal weather data op met publieke API Open-Meteo**

Test eerst of de API werkt, en daarna verpak je het weer in een functie `request_weather(latitude: int, longitude: int, date: str)`.

In [47]:
ROTTERDAM_COORDINATES = (51.92, 4.47)
TEST_DATE = "2025-10-01"

In [48]:
# Simpel:
def request_weather(latitude: int, longitude: int, date: str):
    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "hourly": "temperature_2m,relative_humidity_2m",
        "timezone": "Europe/Amsterdam"
    }
    response = requests.get(base_url, params=params)
    return response.json()

weather_data = request_weather(ROTTERDAM_COORDINATES[0], ROTTERDAM_COORDINATES[1], TEST_DATE)

print(f"Weather on {TEST_DATE}: {weather_data['hourly']['temperature_2m'][0]}°C, {weather_data['hourly']['relative_humidity_2m'][0]}% humidity")

Weather on 2025-10-01: 11.0°C, 94% humidity


In [49]:
# Complexer met error handling en keuze endpoint

def request_weather(lat_long_coordinates: tuple, date: str):
    """Connect naar Open-Meteo API voor uurlijkse weer data"""
    # Parse date string en vergelijk met vandaag
    request_date = datetime.strptime(date, '%Y-%m-%d').date()
    today = datetime.now().date()
    
    # Kies juiste endpoint: archive voor verleden, forecast voor vandaag/toekomst
    if request_date < today:
        base_url = "https://archive-api.open-meteo.com/v1/archive"
    else:
        base_url = "https://api.open-meteo.com/v1/forecast"
        
    params = {
        "latitude": lat_long_coordinates[0],
        "longitude": lat_long_coordinates[1],
        "start_date": date,
        "end_date": date,
        "hourly": "temperature_2m,relative_humidity_2m",
        "timezone": "Europe/Amsterdam"
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request gefaald: {e}")
        return None

# Test API connection
weather_data = request_weather(ROTTERDAM_COORDINATES, TEST_DATE)
if weather_data and 'hourly' in weather_data:
    print(f"Uurlijkse data voor {TEST_DATE}:")
    print(f"  Eerste uur (00:00): {weather_data['hourly']['temperature_2m'][0]}°C, {weather_data['hourly']['relative_humidity_2m'][0]}%")
    print(f"  Totaal {len(weather_data['hourly']['time'])} uur beschikbaar")

Uurlijkse data voor 2025-10-01:
  Eerste uur (00:00): 11.0°C, 94%
  Totaal 24 uur beschikbaar


## 3. State Management - Hoe houd je bij wat er is veranderd?

Definieer 2 functies:
- `load_state()`, dat de laatste pipeline status inlaadt, of initialiseert bij de eerste keer
- `save_state()`, voor als de pipeline klaar is, om de status te updaten.

Sla op welke files al verwerkt zijn en de timestamp van de laatste run.

In [50]:
STATE_FILE = "pipeline_state.json"

def load_state():
    """Laad de laatste pipeline state"""
    try:
        with open(STATE_FILE, 'r') as f:
            state = json.load(f)
            print(f"State geladen: laatste run op {state['last_run']}")
            print(f"Verwerkte files: {len(state['processed_files'])}")
            return state
    except FileNotFoundError:
        print("Geen eerdere state gevonden, start vanaf nu")
        return {
            "last_run": None,
            "records_processed": 0,
            "processed_files": []
        }

def save_state(state):
    """Bewaar de huidige pipeline state"""
    with open(STATE_FILE, 'w') as f:
        json.dump(state, f, indent=2)
    print("State opgeslagen")

# Load current state
pipeline_state = load_state()

# Filter nieuwe files (die nog niet verwerkt zijn)
new_files = [f for f in files_to_process if f.name not in pipeline_state['processed_files']]
print(f"\nNieuwe files om te verwerken: {len(new_files)}")

State geladen: laatste run op 2025-10-17T00:13:12.400471
Verwerkte files: 2

Nieuwe files om te verwerken: 0


## 4. Data Extraction - Hoe extraheer je de data?

Haal data op uit 3 verschillende bronnen (batch processing).

In [55]:
from pandas import NaT

def extract_sensor_data(json_files):
    """Lees sensor readings uit JSON files"""
    all_data = []
    
    for file_path in json_files:
        with open(file_path, 'r') as f:
            data = json.load(f)
            all_data.extend(data)
            print(f"Gelezen: {file_path.name} ({len(data)} records)")
    
    return all_data

def extract_sensor_metadata(csv_file="sample_data/sensor_metadata.csv"):
    """Lees sensor metadata uit CSV"""
    metadata_df = pd.read_csv(csv_file)
    print(f"Metadata geladen: {len(metadata_df)} sensoren")
    return metadata_df

def extract_weather_data_for_dates(dates):
    """Haal uurlijkse weer data op voor alle unieke datums in de sensor data"""
    weather_by_hour = {}
    
    for date in dates:
        if isinstance(date, type(NaT)):
            continue
        date_str = date.strftime('%Y-%m-%d')
        weather_data = request_weather(ROTTERDAM_COORDINATES, date_str)
        
        if weather_data and 'hourly' in weather_data:
            # Converteer naar dict met timestamp als key
            for i, timestamp in enumerate(weather_data['hourly']['time']):
                weather_by_hour[timestamp] = {
                    "outdoor_temp": weather_data['hourly']['temperature_2m'][i],
                    "outdoor_humidity": weather_data['hourly']['relative_humidity_2m'][i]
                }
            print(f"Weer data voor {date_str}: {len(weather_data['hourly']['time'])} uur opgehaald")
    
    return weather_by_hour

# Extract data uit alle bronnen
raw_sensor_data = extract_sensor_data(new_files if new_files else files_to_process[:1])
sensor_metadata = extract_sensor_metadata()

# Voorbeelden
print(f"Voorbeeld ruwe data:")
print(json.dumps(raw_sensor_data[:2], indent=2))

# Bepaal unieke datums uit sensor data
df_temp = pd.DataFrame(raw_sensor_data)
df_temp['timestamp'] = pd.to_datetime(df_temp['timestamp'])
unique_dates = df_temp['timestamp'].dt.date.unique()
print(f"\nUnieke datums in sensor data: {len(unique_dates)}")

# Ophalen van de weer databron
weather_by_hour = extract_weather_data_for_dates(unique_dates)

Gelezen: sensor_readings_day2.json (21 records)
Metadata geladen: 3 sensoren
Voorbeeld ruwe data:
[
  {
    "sensor_id": "TEMP_001",
    "timestamp": "2025-10-15T08:00:00",
    "value": 19.2,
    "unit": "celsius",
    "location": "room_A101",
    "battery_level": 83
  },
  {
    "sensor_id": "TEMP_002",
    "timestamp": "2025-10-15T08:00:00",
    "value": 20.8,
    "unit": "celsius",
    "location": "room_B203",
    "battery_level": 88
  }
]

Unieke datums in sensor data: 2
Weer data voor 2025-10-15: 24 uur opgehaald


In [ ]:
result = df_temp.merge(sensor_metadata[['sensor_id', 'sensor_type', 'manufacturer', 'model']], on="sensor_id", how="left")
# weather_by_hour
result['timestamp'] = pd.to_datetime(result['timestamp'])
result["timestamp_hour"] = result['timestamp'].dt.floor('h').dt.strftime('%Y-%m-%dT%H:%M')
result["outdoor_temp"] = result['timestamp_hour'].map(lambda h: weather_by_hour.get(h, {}).get('outdoor_temp'))
result["humidity"] = result['timestamp_hour'].map(lambda h: weather_by_hour.get(h, {}).get('outdoor_humidity'))
result = result.drop(columns=["timestamp_hour"])




  sensor_id           timestamp  value     unit   location  battery_level  \
0  TEMP_001 2025-10-15 08:00:00   19.2  celsius  room_A101             83   
1  TEMP_002 2025-10-15 08:00:00   20.8  celsius  room_B203             88   
2   CO2_001 2025-10-15 08:00:00  410.0      ppm  hallway_C             74   
3  TEMP_001 2025-10-15 09:00:00   21.1  celsius  room_A101             83   
4  TEMP_002 2025-10-15 09:00:00   22.9  celsius  room_B203             88   

  sensor_type    manufacturer   model  outdoor_temp  humidity  
0        TEMP      SensorCorp  TC-500          12.4      94.0  
1        TEMP      SensorCorp  TC-500          12.4      94.0  
2         CO2  AirQuality Inc  AQ-200          12.4      94.0  
3        TEMP      SensorCorp  TC-500          12.6      94.0  
4        TEMP      SensorCorp  TC-500          12.6      94.0  


## 5. Transformation - Moet je filteren, omvormen voordat data naar target gaat?

Transformeer en verrijk de data:
- Join sensor readings met metadata
- Voeg weather context toe
- Bereken afgeleide velden
- Filter onrealistische waarden

In [56]:
def transform_sensor_data(raw_data, metadata_df, weather_by_hour):
    """Transformeer en verrijk sensor data"""
    # Converteer naar DataFrame
    df = pd.DataFrame(raw_data)

    # 1. Timestamp conversie
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 2. Join met metadata
    df = df.merge(metadata_df[['sensor_id', 'sensor_type', 'manufacturer', 'model']], 
                   on='sensor_id', how='left')

    # 3. Voeg weer context toe per uur
    # Match sensor timestamp naar dichtstbijzijnde uur voor weather data
    if weather_by_hour:
        df['timestamp_hour'] = df['timestamp'].dt.floor('h').dt.strftime('%Y-%m-%dT%H:%M')
        df['outdoor_temp'] = df['timestamp_hour'].map(lambda h: weather_by_hour.get(h, {}).get('outdoor_temp'))
        df['outdoor_humidity'] = df['timestamp_hour'].map(lambda h: weather_by_hour.get(h, {}).get('outdoor_humidity'))
        df = df.drop(columns=['timestamp_hour'])    

    # 4. Voeg afgeleide kolommen toe
    df['hour_of_day'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['is_working_hours'] = df['hour_of_day'].between(8, 18) & df['day_of_week'].between(0, 4)

    # 5. Filter onrealistische waarden (ETLT - filtering in extract fase)
    df_clean = df[
        ((df['sensor_type'] == 'TEMP') & (df['value'].between(MIN_TEMP, MAX_TEMP))) |
        ((df['sensor_type'] == 'CO2') & (df['value'].between(300, 5000)))
    ].copy()

    # 6. Filter lage batterij (waarschuwing)
    low_battery = df_clean[df_clean['battery_level'] < 20]
    if len(low_battery) > 0:
        print(f"[WAARSCHUWING] {len(low_battery)} readings met lage batterij:")
        for _, row in low_battery.iterrows():
            print(f"  {row['sensor_id']} op {row['timestamp']}: {row['battery_level']}%")
    
    removed = len(df) - len(df_clean)
    print(f"\n{len(df)} records getransformeerd")
    print(f"{removed} onrealistische waarden verwijderd")
    print(f"{len(df_clean)} records behouden")
    
    
    return df_clean

# Transform
transformed_data = transform_sensor_data(raw_sensor_data, sensor_metadata, weather_by_hour)
print("\nGetransformeerde data (eerste 6 rijen):")
transformed_data.head(6)


21 records getransformeerd
2 onrealistische waarden verwijderd
19 records behouden

Getransformeerde data (eerste 6 rijen):


,sensor_id,timestamp,value,unit,location,battery_level,sensor_type,manufacturer,model,outdoor_temp,outdoor_humidity,hour_of_day,day_of_week,is_working_hours
0,TEMP_001,2025-10-15 08:00:00,19.2,celsius,room_A101,83,TEMP,SensorCorp,TC-500,12.4,94.0,8.0,2.0,True
1,TEMP_002,2025-10-15 08:00:00,20.8,celsius,room_B203,88,TEMP,SensorCorp,TC-500,12.4,94.0,8.0,2.0,True
2,CO2_001,2025-10-15 08:00:00,410.0,ppm,hallway_C,74,CO2,AirQuality Inc,AQ-200,12.4,94.0,8.0,2.0,True
3,TEMP_001,2025-10-15 09:00:00,21.1,celsius,room_A101,83,TEMP,SensorCorp,TC-500,12.6,94.0,9.0,2.0,True
4,TEMP_002,2025-10-15 09:00:00,22.9,celsius,room_B203,88,TEMP,SensorCorp,TC-500,12.6,94.0,9.0,2.0,True
5,CO2_001,2025-10-15 09:00:00,680.0,ppm,hallway_C,74,CO2,AirQuality Inc,AQ-200,12.6,94.0,9.0,2.0,True


## 6. Validation and Data Quality - Hoe zorg je ervoor dat je data correct is?

**6 Voer data quality checks uit voordat we data opslaan. Wat is je tolerantieniveau?**

- Check 1: Geen null waarden in kritieke kolommen
- Check 2: Duplicaten
- Check 3: Data coverage - elke sensor heeft data?
- Check 4: Waarde ranges per sensor type
- Check 5: Tijdsgaten (missende metingen)

Tot slot:
- Print rapport
- Sla resultaat van kritische checks op in variabele is_valid (True/False)

In [57]:
len(transformed_data[transformed_data.duplicated(subset=["sensor_id", "timestamp"])])

0

In [12]:
def validate_data(df):
    """Valideer data kwaliteit en return quality rapport"""
    issues = []
    warnings = []
    
    # Check 1: Geen null waarden in kritieke kolommen
    critical_cols = ['sensor_id', 'timestamp', 'value']
    null_counts = df[critical_cols].isnull().sum()
    if null_counts.any():
        issues.append(f"Null waarden gevonden: {null_counts[null_counts > 0].to_dict()}")
        # Toon rijen met null waarden in kritieke kolommen
        for col in critical_cols:
            null_rows = df[df[col].isnull()]
            if len(null_rows) > 0:
                print(f"\n[DEBUG] Rijen met null in '{col}':")
                print(null_rows[critical_cols].to_string())
    
    # Check 2: Duplicaten
    duplicates = df.duplicated(subset=['sensor_id', 'timestamp']).sum()
    if duplicates > 0:
        warnings.append(f"{duplicates} duplicate records gevonden (zullen worden verwijderd)")
    
    # Check 3: Data coverage - elke sensor heeft data?
    sensor_counts = df.groupby('sensor_id').size()
    low_coverage = sensor_counts[sensor_counts < 5]
    if len(low_coverage) > 0:
        warnings.append(f"Lage data coverage voor: {low_coverage.to_dict()}")
    
    # Check 4: Waarde ranges per sensor type
    temp_out_of_range = df[(df['sensor_type'] == 'TEMP') & ~df['value'].between(15, 30)]
    co2_high = df[(df['sensor_type'] == 'CO2') & (df['value'] > 1000)]
    
    if len(temp_out_of_range) > 0:
        warnings.append(f"{len(temp_out_of_range)} temperatuur waarden buiten comfortzone (15-30 graden C)")
    if len(co2_high) > 0:
        warnings.append(f"{len(co2_high)} CO2 waarden > 1000 ppm (slechte luchtkwaliteit!)")
    
    # Check 5: Tijdsgaten (missende metingen)
    df_sorted = df.sort_values(['sensor_id', 'timestamp'])
    for sensor_id in df['sensor_id'].unique():
        sensor_data = df_sorted[df_sorted['sensor_id'] == sensor_id]
        if len(sensor_data) > 1:
            time_diffs = sensor_data['timestamp'].diff()
            large_gaps = time_diffs[time_diffs > timedelta(hours=1)]
            if len(large_gaps) > 0:
                warnings.append(f"{sensor_id}: {len(large_gaps)} tijdsgaten > 1 uur")
    
    # Print rapport
    print("\n" + "="*60)
    print("DATA QUALITY RAPPORT")
    print("="*60)
    
    if not issues and not warnings:
        print("[OK] Alle validaties geslaagd!")
        return True
    
    if issues:
        print("\n[KRITIEKE ISSUES]")
        for issue in issues:
            print(f"  - {issue}")
    
    if warnings:
        print("\n[WAARSCHUWINGEN]")
        for warning in warnings:
            print(f"  - {warning}")
    
    # Tolerantieniveau: geen kritieke issues, waarschuwingen zijn OK
    is_valid = len(issues) == 0
    print(f"\n[INFO] Data {'geaccepteerd' if is_valid else 'AFGEKEURD'}")
    print(f"Tolerantieniveau: geen kritieke issues toegestaan")
    
    return is_valid

# Validate
is_valid = validate_data(transformed_data)


[DEBUG] Rijen met null in 'timestamp':
   sensor_id timestamp  value
17   CO2_001       NaT  760.0

DATA QUALITY RAPPORT

[KRITIEKE ISSUES]
  - Null waarden gevonden: {'timestamp': 1}

[WAARSCHUWINGEN]
  - 3 CO2 waarden > 1000 ppm (slechte luchtkwaliteit!)
  - TEMP_001: 9 tijdsgaten > 1 uur
  - TEMP_002: 9 tijdsgaten > 1 uur
  - CO2_001: 8 tijdsgaten > 1 uur

[INFO] Data AFGEKEURD
Tolerantieniveau: geen kritieke issues toegestaan


## 7. Data Loading - Waar en hoe sla je de data op?

Schrijf de gevalideerde data naar de target database/storage.

**Parquet vs CSV:**
- CSV: Row-based, makkelijk leesbaar, groot
- Parquet: Column-based, gecomprimeerd, snel voor analytics

In [13]:
is_valid = True  # Forceer validatie voor demo

In [14]:
OUTPUT_FILE = "sensor_data_warehouse.csv"
OUTPUT_PARQUET = "sensor_data_warehouse.parquet"

def load_data(df):
    """Laad data naar target (CSV en Parquet)"""
    # In productie: df.to_sql('sensor_readings', con=db_engine, if_exists='append')
    
    try:
        # Append to existing file
        existing_df = pd.read_csv(OUTPUT_FILE)
        existing_df['timestamp'] = pd.to_datetime(existing_df['timestamp'])
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        print(f"Bestaande data gevonden: {len(existing_df)} records")
    except FileNotFoundError:
        combined_df = df
        print("Nieuwe data warehouse aangemaakt")
    
    # Verwijder duplicaten
    before_dedup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['sensor_id', 'timestamp'])
    after_dedup = len(combined_df)
    
    print(f"{before_dedup - after_dedup} duplications verwijderd")
    
    # Save als CSV (row-based)
    combined_df.to_csv(OUTPUT_FILE, index=False)
    csv_size = Path(OUTPUT_FILE).stat().st_size / 1024  # KB
    
    # Save als Parquet (column-based, beter voor analytics)
    combined_df.to_parquet(OUTPUT_PARQUET, index=False, engine='pyarrow')
    parquet_size = Path(OUTPUT_PARQUET).stat().st_size / 1024  # KB
    
    print(f"\n{len(df)} nieuwe records opgeslagen")
    print(f"Totaal in warehouse: {len(combined_df)} records")
    print(f"\nFile sizes:")
    print(f"  CSV: {csv_size:.1f} KB")
    print(f"  Parquet: {parquet_size:.1f} KB")

    if parquet_size > csv_size:
        print("\n[WAARSCHUWING] Parquet bestand is groter dan CSV. " \
        "Rule-of-thumb: gebruik CSV bij minder dan ≈1,000 records.")
    
    return len(df)

# Load (alleen als validatie OK is)
if is_valid:
    records_loaded = load_data(transformed_data)
else:
    print("[FOUT] Data niet geladen vanwege validatie fouten")
    records_loaded = 0

Bestaande data gevonden: 52 records
52 duplications verwijderd

52 nieuwe records opgeslagen
Totaal in warehouse: 52 records

File sizes:
  CSV: 5.3 KB
  Parquet: 9.1 KB

[WAARSCHUWING] Parquet bestand is groter dan CSV. Rule-of-thumb: gebruik CSV bij minder dan ≈1,000 records.


## 8. Archiving and Retention - Hoe lang bewaar je data?

Implementeer een retention policy: verwijder data ouder dan X dagen.

In [15]:
def apply_retention_policy(retention_days=90):
    """Verwijder data ouder dan retention period"""
    try:
        df = pd.read_csv(OUTPUT_FILE)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        cutoff_date = datetime.now() - timedelta(days=retention_days)
        
        # Filter data binnen retention period
        df_retained = df[df['timestamp'] >= cutoff_date]
        df_archived = df[df['timestamp'] < cutoff_date]
        
        if len(df_archived) > 0:
            # In productie: verplaats naar archive storage (S3 Glacier, etc.)
            archive_file = f"archive_{cutoff_date.date()}.csv"
            df_archived.to_csv(archive_file, index=False)
            df_retained.to_csv(OUTPUT_FILE, index=False)
            
            print(f"{len(df_archived)} oude records gearchiveerd naar {archive_file}")
            print(f"{len(df_retained)} records behouden (< {retention_days} dagen oud)")
            print(f"Oudste record: {df_retained['timestamp'].min()}")
            print(f"Nieuwste record: {df_retained['timestamp'].max()}")
        else:
            print(f"Alle data binnen retention period ({retention_days} dagen)")
            print(f"Oudste record: {df['timestamp'].min()}")
            
    except FileNotFoundError:
        print("Geen data om te archiveren")

# Apply retention (90 dagen)
apply_retention_policy(retention_days=90)

Alle data binnen retention period (90 dagen)
Oudste record: 2025-10-14 08:00:00


## 9. Pipeline Afronden - Update State

Sla de nieuwe pipeline state op voor de volgende run.

In [16]:
# Update pipeline state
if is_valid:
    pipeline_state['last_run'] = datetime.now().isoformat()
    pipeline_state['records_processed'] += records_loaded
    
    # Voeg verwerkte files toe
    new_file_names = [f.name for f in (new_files if new_files else files_to_process[:1])]
    pipeline_state['processed_files'].extend(new_file_names)
    
    save_state(pipeline_state)

print("\n" + "="*60)
print("PIPELINE SUCCESVOL AFGEROND" if is_valid else "PIPELINE AFGEBROKEN")
print("="*60)
print(f"Totaal verwerkte records deze run: {records_loaded}")
print(f"Totaal verwerkte records (lifetime): {pipeline_state['records_processed']}")
print(f"Totaal verwerkte files: {len(pipeline_state['processed_files'])}")

State opgeslagen

PIPELINE SUCCESVOL AFGEROND
Totaal verwerkte records deze run: 52
Totaal verwerkte records (lifetime): 52
Totaal verwerkte files: 2


## 10. Check nieuwe state
Run nu nog een keer de code van het blok "3. State Management" en je ziet dat er geen nieuwe files meer te verwerken zijn.